In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [ ]:
import pathlib
import warnings 
warnings.filterwarnings('ignore') # Supress warnings

IMG_SIZE = 224
NUM_CLASSES = 10

# Path con subdirectorios para cada label (0's, 1's, ...)
data_dir = pathlib.Path('../data/MNIST/')

classnames = dict((f'{i}', i) for i in range(NUM_CLASSES))

In [ ]:
img_height,img_width=IMG_SIZE,IMG_SIZE
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical'
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(6):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(classnames[str(labels.numpy()[i])])
        plt.axis("off")

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                    input_shape=(IMG_SIZE,IMG_SIZE,3),
                    pooling='avg',
                    classes=NUM_CLASSES,
                    weights='imagenet'
                    )
for layer in pretrained_model.layers:
    layer.trainable=False

resnet_model.add(pretrained_model)

# add FC layer and output layer
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
L_R = 0.001
N_EPOCHS = 10
resnet_model.summary()

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
resnet_model.compile(optimizer=Adam(lr=L_R),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=N_EPOCHS)

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
resnet_model.save_weights('./resnetUTK')

In [ ]:
resnet_model = Sequential()
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                    input_shape=(IMG_SIZE,IMG_SIZE,3),
                    pooling='avg',
                    classes=NUM_CLASSES,
                    weights='imagenet'
                    )
for layer in pretrained_model.layers:
    layer.trainable=False
resnet_model.add(pretrained_model)
# add FC layer and output layer
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(NUM_CLASSES, activation='softmax'))
#load weights
model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
import cv2
image=cv2.imread(str(roses[0]))
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)

In [ ]:
pred=resnet_model.predict(image)
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)